In [13]:
# Importing the required library
import pandas as pd

# Reading the Excel file into a Pandas DataFrame
IV = pd.read_excel('Data/Data_BA_modified.xlsx')

# Displaying the first 5 rows of the DataFrame to verify that the data has been read correctly
IV

,Område,Team,Indsatslængde,Indsatsparagraf,Leverandør,PrisIAlt,Haendelsesdato,alder,alder_ved_hændelse,sag_start,...,BaggrundBarnet_BaggrundBarnet_Anden form for omsorgssvigt over for barn/ung,BaggrundBarnet_BaggrundBarnet_Betydelig eller varigt nedsat fysisk eller psykisk funktionsevne hos barn/ung,BaggrundBarnet_BaggrundBarnet_Overgreb mod barn/ung f.eks. seksuelt eller voldeligt,BaggrundHjemmet_BaggrundHjemmet_Anden bekymrende adfærd hos forældre,BaggrundHjemmet_BaggrundHjemmet_Betydelig eller varigt nedsat fysisk eller psykisk funktionsevne hos forældre,BaggrundHjemmet_BaggrundHjemmet_Højt konfliktniveau eller vold i hjemmet mellem voksne,BaggrundHjemmet_BaggrundHjemmet_Misbrug hos forældre,Underretter_Anden_Kommune,Underretter_Sundhedsvæsen,Underretter_Socialrådgiver
0,BBU City-Østerbro,BBU City-Østerbro - MY Børnegruppen,90,"SEL § 52, stk. 3, nr. 3 (fx familiebehandling)",Foranstaltningshus BBU City-Østerbro,28091,2020-07-14,6,3,2020-07-14,...,0,0,0,0,0,0,0,0,0,0
1,BBU City-Østerbro,BBU City-Østerbro - MY Børnegruppen,161,"SEL § 52, stk. 3, nr. 3 (fx familiebehandling)",Foranstaltningshus BBU City-Østerbro,47375,2020-09-18,2,0,2020-09-18,...,0,0,0,0,0,0,0,1,0,0
2,BBU City-Østerbro,BBU City-Østerbro - MY Børnegruppen,0,"SEL § 52, stk. 3, nr. 3 (fx familiebehandling)",Børne- og Familiehuset Arild,39,2021-05-21,1,0,2021-05-21,...,1,0,0,1,0,0,0,0,1,0
3,BBU Valby-Vesterbro-Kgs.Enghave,BBU Valby-Vesterbro-Kgs.Enghave - Styrket modt...,239,"SEL § 52, stk. 3, nr. 3 (fx familiebehandling)",Foranstaltningshus BBU Valby-Vesterbro-Kgs.Eng...,141911,2020-03-05,2,0,2020-03-05,...,0,0,0,0,0,0,1,0,0,0
4,BBU AF Økonomi og Digitalisering,NaN,123,"SEL § 52, stk. 3, nr. 3 (behandling af barn/un...",Psykolog Tanja Skov Rønne,49000,2021-01-23,5,3,2021-01-23,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
646,BBU Valby-Vesterbro-Kgs.Enghave,BBU Valby-Vesterbro-Kgs.Enghave - Børneliv,134,"SEL § 52, stk. 3, nr. 3 (fx familiebehandling)",Mentrax Consult ApS,14880,2020-11-30,7,4,2020-11-30,...,0,0,0,0,0,0,0,0,0,0
647,BBU Valby-Vesterbro-Kgs.Enghave,BBU Valby-Vesterbro-Kgs.Enghave - Børneliv,182,"SEL § 52, stk. 3, nr. 3 (fx familiebehandling)",Memox & Øresund Psykoterapi ApS,61639,2023-06-23,5,5,2022-04-08,...,0,0,0,0,0,0,0,0,0,0
648,BBU Valby-Vesterbro-Kgs.Enghave,BBU Valby-Vesterbro-Kgs.Enghave - Børneliv,112,"SEL § 52, stk. 3, nr. 3 (fx familiebehandling)",Foranstaltningshus BBU Nørrebro-Bispebjerg,36903,2020-07-02,6,2,2020-07-02,...,0,0,0,0,0,0,0,0,0,0
649,BBU Valby-Vesterbro-Kgs.Enghave,BBU Valby-Vesterbro-Kgs.Enghave - Børneliv,899,"SEL § 52, stk. 3, nr. 3 (fx familiebehandling)",Foranstaltningshus BBU Valby-Vesterbro-Kgs.Eng...,101450,2020-03-23,7,3,2020-03-23,...,0,0,0,0,0,0,0,0,0,0


In [14]:
# Group by 'Område' and compute the average price for each
average_price_by_område = IV.groupby('Område')['PrisIAlt'].mean().sort_values(ascending=False)

average_price_by_område

Område
Teamet for koordineret indsats (KIT)    149240.000000
BBU Valby-Vesterbro-Kgs.Enghave          98769.597015
BCH Børneområdet                         97714.956522
BBU Brønshøj-Husum-Vanløse               94237.909091
BBU City-Østerbro                        93842.218182
BBU Amager                               71151.040816
BBU Nørrebro-Bispebjerg                  65864.847716
BBU AF Økonomi og Digitalisering         49000.000000
BBU Flygtningeindsats                    27722.000000
Name: PrisIAlt, dtype: float64

In [15]:
import statsmodels.api as sm

# Create dummy variables for the 'Område' column
data = pd.concat([IV, pd.get_dummies(IV['Område'], drop_first=True, prefix='Område')], axis=1)

# Extract control variables
control_columns = [col for col in IV.columns if col.startswith(("Underretter", "BaggrundBarnet", "BaggrundHjemmet"))]

# Exogenous variables for the first stage (instrument + controls)
exog_vars = control_columns + [col for col in IV.columns if col.startswith("Område_")]

# First stage regression: Regress PrisIAlt on the instrument and control variables
X_first_stage = sm.add_constant(IV[exog_vars])
first_stage_model = sm.OLS(data['PrisIAlt'], X_first_stage).fit()

# Output the summary of the first stage regression
first_stage_model.summary()


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               PrisIAlt   R-squared:                       0.080
Model:                            OLS   Adj. R-squared:                  0.014
Method:                 Least Squares   F-statistic:                     1.222
Date:                Fri, 20 Oct 2023   Prob (F-statistic):              0.162
Time:                        11:08:07   Log-Likelihood:                -8465.5
No. Observations:                 651   AIC:                         1.702e+04
Df Residuals:                     607   BIC:                         1.722e+04
Df Model:                          43                                         
Covariance Type:            nonrobust                                         
========================================================================================================================================================================================
                                                                                                                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
const                                                                                                                  9.99e+04   1.37e+04      7.314      0.000    7.31e+04    1.27e+05
Underretter_(Ingen)                                                                                                   2.584e+04   7.61e+04      0.340      0.734   -1.24e+05    1.75e+05
Underretter_Anbringelsessted                                                                                          1.119e+05    7.6e+04      1.473      0.141   -3.73e+04    2.61e+05
Underretter_Anden offentlig myndighed                                                                                -1.245e+04    2.3e+04     -0.541      0.589   -5.77e+04    3.28e+04
Underretter_Anonym                                                                                                    2.824e+04   2.02e+04      1.395      0.163   -1.15e+04     6.8e+04
Underretter_Barnets netværk                                                                                           1.278e+04   4.51e+04      0.284      0.777   -7.57e+04    1.01e+05
Underretter_Daginstitution/Dagpleje                                                                                  -4277.1601   1.41e+04     -0.303      0.762    -3.2e+04    2.34e+04
Underretter_Familie                                                                                                  -1.915e+04   3.47e+04     -0.552      0.581   -8.72e+04    4.89e+04
Underretter_Familiens bekendtskabskreds                                                                              -5938.2391   7.67e+04     -0.077      0.938   -1.57e+05    1.45e+05
Underretter_Forældremyndighedsindehaver                                                                              -2.534e+04   2.31e+04     -1.098      0.272   -7.06e+04       2e+04
Underretter_Krisecenter                                                                                               6397.8112   2.01e+04      0.318      0.751   -3.32e+04     4.6e+04
Underretter_Naboer                                                                                                   -2.842e+04   4.96e+04     -0.573      0.567   -1.26e+05    6.89e+04
Underretter_Politi                                                                                                   -1.997e+04   1.98e+04     -1.010      0.313   -5.88e+04    1.89e+04
Underretter_Psykiatri                                                                                                 9469.

In [16]:
# First Stage: Predicted values are already available as 'predicted_price'

# Second Stage: Regress the dependent variable on the predicted values from the first stage and controls
X_second_stage = sm.add_constant(pd.concat([IV['predicted_price'], controls], axis=1))
y = IV['sag_varighed']

second_stage = sm.OLS(y, X_second_stage).fit()

second_stage.summary()


KeyError: 'predicted_price'